In [32]:
import torch
from transformers import Data2VecTextForQuestionAnswering, AutoTokenizer
from PIL import Image
import requests

In [33]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [34]:
version = "facebook/data2vec-text-base"
question, text = "Who was Jim Henson?", "Jim Henson was a nice puppet"

# AutoTokenizer

In [35]:
tokenizer: AutoTokenizer = AutoTokenizer.from_pretrained(version)
tokenizer

RobertaTokenizerFast(name_or_path='facebook/data2vec-text-base', vocab_size=50265, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'sep_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'cls_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True)}, clean_up_tokenization_spaces=True)

## special ids and tokens

In [36]:
print(tokenizer.all_special_ids)
print(tokenizer.all_special_tokens)

[0, 2, 3, 1, 50264]
['<s>', '</s>', '<unk>', '<pad>', '<mask>']


## processor

In [37]:
inputs = tokenizer(
    question,
    text,
    return_tensors="pt",  # 返回数据格式 np pt tf jax
    padding=True,  # 填充方式选择 [True, 'longest', 'max_length', 'do_not_pad']
    # max_length = max_length,  # 如果使用max_length要将padding设置为 "max_length"
    add_special_tokens=True,
).to(device, torch.float16)
inputs

{'input_ids': tensor([[    0, 12375,    21,  2488,   289, 13919,   116,     2,     2, 24021,
           289, 13919,    21,    10,  2579, 29771,     2]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [38]:
inputs["input_ids"]

tensor([[    0, 12375,    21,  2488,   289, 13919,   116,     2,     2, 24021,
           289, 13919,    21,    10,  2579, 29771,     2]], device='cuda:0')

# Data2VecTextForQuestionAnswering

Data2VecText Model with a span classification head on top for extractive question-answering tasks like SQuAD (a linear layers on top of the hidden-states output to compute span start logits and span end logits).

In [39]:
model: Data2VecTextForQuestionAnswering = (
    Data2VecTextForQuestionAnswering.from_pretrained(
        version, torch_dtype=torch.float16
    ).to(device)
)
model

Some weights of Data2VecTextForQuestionAnswering were not initialized from the model checkpoint at facebook/data2vec-text-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Data2VecTextForQuestionAnswering(
  (data2vec_text): Data2VecTextModel(
    (embeddings): Data2VecTextForTextEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): Data2VecTextEncoder(
      (layer): ModuleList(
        (0-11): 12 x Data2VecTextLayer(
          (attention): Data2VecTextAttention(
            (self): Data2VecTextSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): Data2VecTextSelfOutput(
              (dense): Linear(in_features

In [40]:
model.eval()
with torch.inference_mode():
    outputs = model(**inputs)
outputs

QuestionAnsweringModelOutput(loss=None, start_logits=tensor([[-0.1217,  0.2920,  0.2913,  0.2985,  0.2927,  0.2820,  0.2944, -0.1047,
          0.2971,  0.2947,  0.2913,  0.2824,  0.2974,  0.2868,  0.2952,  0.2872,
         -0.1079]], device='cuda:0'), end_logits=tensor([[-0.2358,  0.0484,  0.0486,  0.0177,  0.0466,  0.0137,  0.0477, -0.2503,
          0.0435,  0.0456,  0.0484,  0.0181,  0.0285,  0.0554,  0.0398,  0.0031,
         -0.2417]], device='cuda:0'), hidden_states=None, attentions=None)

In [41]:
answer_start_index = outputs.start_logits.argmax()
answer_start_index

tensor(3, device='cuda:0')

In [42]:
answer_end_index = outputs.end_logits.argmax()
answer_end_index

tensor(13, device='cuda:0')

In [43]:
predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
predict_answer_tokens

tensor([ 2488,   289, 13919,   116,     2,     2, 24021,   289, 13919,    21,
           10], device='cuda:0')

In [44]:
tokenizer.decode(predict_answer_tokens)

' Jim Henson?</s></s>Jim Henson was a'